In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
#nltk.download('punkt')
lemmatizer = WordNetLemmatizer()
import json
import pickle
intents_file = open('intents.json').read()
intents = json.loads(intents_file)

In [2]:
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

for intent in intents['intents']:

    for pattern in intent['patterns']:

        #tokenize each word

        word = nltk.word_tokenize(pattern)

        words.extend(word)        

        #add documents in the corpus

        documents.append((word, intent['tag']))

        # add to our classes list

        if intent['tag'] not in classes:

            classes.append(intent['tag'])

print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'm

In [3]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
87 unique lemmatized words ["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [4]:
# create the training data
training = []

# create empty array for the output
output_empty = [0] * len(classes)

# training set, bag of words for every sentence
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    word_patterns = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle the features and make numpy array
random.shuffle(training)
training = np.array(training)
# create training and testing lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data is created")

Training data is created


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [5]:
# deep neural networds model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#Training and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model is created")

Epoch 1/200
10/10 [==============================] - 0s 887us/step - loss: 2.2586 - accuracy: 0.0568
Epoch 2/200
10/10 [==============================] - 0s 894us/step - loss: 2.2026 - accuracy: 0.1516
Epoch 3/200
10/10 [==============================] - 0s 722us/step - loss: 2.0871 - accuracy: 0.1789
Epoch 4/200
10/10 [==============================] - 0s 664us/step - loss: 2.0158 - accuracy: 0.2269
Epoch 5/200
10/10 [==============================] - 0s 682us/step - loss: 1.9848 - accuracy: 0.3217
Epoch 6/200
10/10 [==============================] - 0s 734us/step - loss: 1.9004 - accuracy: 0.4651
Epoch 7/200
10/10 [==============================] - 0s 782us/step - loss: 1.7433 - accuracy: 0.4627
Epoch 8/200
10/10 [==============================] - 0s 758us/step - loss: 1.6954 - accuracy: 0.4751
Epoch 9/200
10/10 [==============================] - 0s 871us/step - loss: 1.4922 - accuracy: 0.6406
Epoch 10/200
10/10 [==============================] - 0s 714us/step - loss: 1.3783 - accura

10/10 [==============================] - 0s 664us/step - loss: 0.0539 - accuracy: 1.0000
Epoch 82/200
10/10 [==============================] - 0s 720us/step - loss: 0.0983 - accuracy: 1.0000
Epoch 83/200
10/10 [==============================] - 0s 670us/step - loss: 0.0615 - accuracy: 0.9941
Epoch 84/200
10/10 [==============================] - 0s 694us/step - loss: 0.0663 - accuracy: 0.9629
Epoch 85/200
10/10 [==============================] - 0s 645us/step - loss: 0.0710 - accuracy: 0.9629
Epoch 86/200
10/10 [==============================] - 0s 697us/step - loss: 0.0593 - accuracy: 1.0000
Epoch 87/200
10/10 [==============================] - 0s 642us/step - loss: 0.0670 - accuracy: 0.9720
Epoch 88/200
10/10 [==============================] - 0s 690us/step - loss: 0.0310 - accuracy: 1.0000
Epoch 89/200
10/10 [==============================] - 0s 630us/step - loss: 0.0373 - accuracy: 0.9892
Epoch 90/200
10/10 [==============================] - 0s 686us/step - loss: 0.0288 - accuracy: 

10/10 [==============================] - 0s 614us/step - loss: 0.0144 - accuracy: 1.0000
Epoch 161/200
10/10 [==============================] - 0s 646us/step - loss: 0.0063 - accuracy: 1.0000
Epoch 162/200
10/10 [==============================] - 0s 651us/step - loss: 0.0221 - accuracy: 1.0000
Epoch 163/200
10/10 [==============================] - 0s 698us/step - loss: 0.0307 - accuracy: 1.0000
Epoch 164/200
10/10 [==============================] - 0s 684us/step - loss: 0.0033 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 658us/step - loss: 0.0194 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 684us/step - loss: 0.0173 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 668us/step - loss: 0.0026 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 666us/step - loss: 0.0189 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 683us/step - loss: 0.0121 - a